In [1]:
#include <vector>
#include <array>
#include <cstdint>
#include <cmath>
#include <iostream>
#include <random>
#include <H5Cpp.h>
#include <chrono>
#include <argparse.hpp>
#include <bhxx/bhxx.hpp>

const double G = 6.673e-11;
const double SOLAR_MASS = 1.98892e30;

/** Class that represent a set of bodies, which can be the sun, planets, and/or asteroids */
class Bodies {
public:
    // The mass of the bodies
    bhxx::BhArray<double> mass;
    // The position of the bodies in the x, y, and z axis
    bhxx::BhArray<double> pos_x;
    bhxx::BhArray<double> pos_y;
    bhxx::BhArray<double> pos_z;
    // The velocity of the bodies in the x, y, and z axis
    bhxx::BhArray<double> vel_x;
    bhxx::BhArray<double> vel_y;
    bhxx::BhArray<double> vel_z;

    /** Return a copy of all bodies */
    Bodies copy() {
        return Bodies{mass.copy(), pos_x.copy(), pos_y.copy(), pos_z.copy(), vel_x.copy(), vel_y.copy(), vel_z.copy()};
    }

};

/** Class that represent a solar system, which consist of a sun, some planets, and many asteroids. */
class SolarSystem {
public:
    Bodies sun_and_planets;
    Bodies asteroids;

    /** Return a copy of the solar system */
    SolarSystem copy() {
       return SolarSystem{sun_and_planets.copy(), asteroids.copy()};
    }
};

/** Function that returns `x` squared for each element in `x` */
bhxx::BhArray<double> squared(const bhxx::BhArray<double> &x) {
    return x * x;
}

/** Function that returns the magnitude of velocity for each element `i` in `pos_x[i], pos_y[i], pos_z[i]` */
const bhxx::BhArray<double> circlev(const bhxx::BhArray<double> &pos_x,
                                    const bhxx::BhArray<double> &pos_y,
                                    const bhxx::BhArray<double> &pos_z) {
    return bhxx::sqrt(G * 1e6 * SOLAR_MASS / bhxx::sqrt(squared(pos_x) + squared(pos_y) + squared(pos_z)));
}


/** Return a new random solar system
 *
 * @param position_limit   The coordinate limit
 * @param num_of_planets   The number of planets
 * @param num_of_asteroids The number of asteroids
 * @param seed             The random seed the use
 * @return                 The new solar system
 */
SolarSystem random_system(double position_limit, uint64_t num_of_planets, uint64_t num_of_asteroids, uint64_t seed) {
    
    
    auto pos_x = bhxx::random.randn<double>({1, num_of_planets+1});// til at lave en random array 0-1
    auto pos_y = bhxx::random.randn<double>({1, num_of_planets+1});
    auto pos_z = bhxx::random.randn<double>({1, num_of_planets+1}) * 0.01;
    
    
    
    auto dist = ( bhxx::random.randn<double>({1, num_of_planets+1}) * 1.0 / bhxx::sqrt(squared(pos_x) + squared(pos_y) + squared(pos_z))) - ( bhxx::random.randn<double>({1, num_of_planets+1})*.8 -  bhxx::random.randn<double>({1, num_of_planets+1}) * .1);
    pos_x *= position_limit * dist * sign( bhxx::random.randn<double>({1, num_of_planets+1})*.5 - bhxx::random.randn<double>({1, num_of_planets+1}));
    pos_y *= position_limit * dist * sign( bhxx::random.randn<double>({1, num_of_planets+1})*.5 - bhxx::random.randn<double>({1, num_of_planets+1}));
    pos_z *= position_limit * dist * sign( bhxx::random.randn<double>({1, num_of_planets+1})*.5 - bhxx::random.randn<double>({1, num_of_planets+1}));
    
    auto magv = circlev(pos_x, pos_y, pos_z);
    auto abs_angle = bhxx::arctan(bhxx::absolute (pos_y / pos_x));

    auto theta_v = M_PI / 2 - abs_angle;
    
    
    auto mass = bhxx::random.randn<double>({1, num_of_planets+1}) * SOLAR_MASS * 10 + 1e20;
    mass[0][0] = 1e6 * SOLAR_MASS;
    
    auto vel_x = bhxx::ones<double>({1, num_of_planets+1})*-1  * bhxx::cos(theta_v) * magv; //mangler signs men ved ikke lige hvordan det skal laves uden et loop
    auto vel_y = bhxx::ones<double>({1, num_of_planets+1}) * bhxx::sin(theta_v) * magv;
    auto vel_z = bhxx::empty<float>({1,num_of_planets+1});
    pos_x[0][0] = 0;
    pos_y[0][0] = 0;
    pos_z[0][0] = 0;
    vel_x[0][0] = 0;
    vel_y[0][0] = 0;
    vel_z[0][0] = 0;


    
    Bodies sun_and_planets{mass,pos_x,pos_y,pos_z,vel_x,vel_y,vel_z};


    
    
    //astroides
    auto apos_x = bhxx::random.randn<double>({1, num_of_asteroids});// til at lave en random array 0-1
    auto apos_y = bhxx::random.randn<double>({1, num_of_asteroids});
    auto apos_z = bhxx::random.randn<double>({1, num_of_asteroids}) * 0.01;
    auto adist = ( bhxx::random.randn<double>({1, num_of_asteroids}) * 1.0 / bhxx::sqrt(squared(apos_x) + squared(apos_y) + squared(apos_z))) - bhxx::random.randn<double>({1, num_of_asteroids})*.2;
    apos_x *= position_limit * adist * sign( bhxx::random.randn<double>({1, num_of_asteroids})*.5 - bhxx::random.randn<double>({1, num_of_asteroids}));
    apos_y *= position_limit * adist * sign( bhxx::random.randn<double>({1, num_of_asteroids})*.5 - bhxx::random.randn<double>({1, num_of_asteroids}));
    apos_z *= position_limit * adist * sign( bhxx::random.randn<double>({1, num_of_asteroids})*.5 - bhxx::random.randn<double>({1, num_of_asteroids}));
    
    auto amagv = circlev(apos_x, apos_y, apos_z);
    auto aabs_angle = bhxx::arctan(bhxx::absolute (apos_y / apos_x));

    auto atheta_v = M_PI / 2 - aabs_angle;
    
    
    auto amass = bhxx::random.randn<double>({1, num_of_asteroids}) * SOLAR_MASS * 10 + 1e14;
    
    auto avel_x = bhxx::random.randn<double>({1, num_of_asteroids});
    auto avel_y = bhxx::random.randn<double>({1, num_of_asteroids});
    auto avel_z = bhxx::empty<float>({1,num_of_asteroids});;
    
    Bodies asteroids{amass,apos_x,apos_y,apos_z,avel_x,avel_y,avel_z};


    
    return SolarSystem{sun_and_planets,asteroids};
}

/** Fill the diagonal of `ary` with the value `val` */
void fill_diagonal(bhxx::BhArray<double> &ary, double val) {
    if (ary.shape().size() != 2 || ary.shape()[0] != ary.shape()[1]) {
        throw std::runtime_error("The array must be a squired matrix");
    }
    if (!ary.isContiguous()) {
        throw std::runtime_error("`ary` must be contiguous");
    }
    uint64_t view_size = ary.shape()[0];
    int64_t view_stride = view_size+1;
    // Create a view that represents the diagonal of `ary`
    bhxx::BhArray<double> diagonal_view(ary.base(), {view_size}, {view_stride}, ary.offset());
    // And assign `val` to the diagonal
    diagonal_view = val;
}

/** Update the velocity of all bodies in `a` based on the bodies in `b`
 *
 * @param a                   The bodies to update
 * @param b                   The bodies which act on `a`
 * @param diagonal_zero_fill  Set the diagonal of the force matrices to zero
 * @param dt                  The time step size
 */
void update_velocity(Bodies &a, const Bodies &b, double dt) {
    // TODO: Implement
    // lav ting til vectore
    // Euclidean distance
    
    bhxx::BhArray<double> r = bhxx::sqrt(squared(b.pos_x - a.pos_x) + squared(b.pos_y - a.pos_y) + squared(b.pos_z - a.pos_z));
    
    // Force:  F = ((G m_a m_b)/r^2)*((x_b-x_a)/r)
    bhxx::BhArray<double> F = (G * a.mass * b.mass / squared(r)); // Force without direction
    
    std::cout << squared(r) ;

    // Update velocity of `a`
    a.vel_x += F * ((b.pos_x - a.pos_x) / r) / a.mass * dt;
    a.vel_y += F * ((b.pos_y - a.pos_y) / r) / a.mass * dt;
    a.vel_z += F * ((b.pos_z - a.pos_z) / r) / a.mass * dt;
    

}


/** Integrate one time step of the solar system
 *
 * @param solar_system  The solar system to update
 * @param dt            The time step size
 */
void integrate(SolarSystem &solar_system, double dt) {
    // TODO: Implement
    // forloops skal kigges igennem helst fjernes
    // Update velocity of the sub and planets
    

    
    update_velocity(solar_system.asteroids, solar_system.asteroids, dt);
    update_velocity(solar_system.sun_and_planets, solar_system.sun_and_planets, dt);
    
    solar_system.sun_and_planets.pos_x += solar_system.sun_and_planets.vel_x * dt;
    solar_system.sun_and_planets.pos_y += solar_system.sun_and_planets.vel_y * dt;
    solar_system.sun_and_planets.pos_z += solar_system.sun_and_planets.vel_z * dt;
    
    
    solar_system.asteroids.pos_x += solar_system.asteroids.vel_x * dt;
    solar_system.asteroids.pos_y += solar_system.asteroids.vel_y * dt;
    solar_system.asteroids.pos_z += solar_system.asteroids.vel_z * dt;
    
    std::cout << solar_system.sun_and_planets.vel_x;
}
/** Write data to a hdf5 file
 *
 * @param group  The hdf5 group to write in
 * @param name   The name of the data
 * @param shape  The shape of the data
 * @param data   The data
 */
void write_hdf5(H5::Group &group, const std::string &name, const std::vector<hsize_t> &shape,
                const std::vector<double> &data) {

    H5::DataSpace dataspace(static_cast<int>(shape.size()), &shape[0]);
    H5::DataSet dataset = group.createDataSet(name.c_str(), H5::PredType::NATIVE_DOUBLE, dataspace);
    dataset.write(&data[0], H5::PredType::NATIVE_DOUBLE);
}

/** Write the solar system to a hdf5 file (use `visual.py` to visualize the hdf5 data)
 *
 * @param solar_systems  The solar system to write
 * @param filename       The filename to write to
 */
void write_hdf5(const std::vector<SolarSystem> &solar_systems, const std::string &filename) {

    H5::H5File file(filename, H5F_ACC_TRUNC);

    for (uint64_t i = 0; i < solar_systems.size(); ++i) {
        H5::Group group(file.createGroup("/" + std::to_string(i)));
        {
            std::vector<double> data;
            for (double elem: solar_systems[i].sun_and_planets.pos_x.vec()) {
                data.push_back(elem);
            }
            for (double elem: solar_systems[i].sun_and_planets.pos_y.vec()) {
                data.push_back(elem);
            }
            for (double elem: solar_systems[i].sun_and_planets.pos_z.vec()) {
                data.push_back(elem);
            }
            write_hdf5(group, "sun_and_planets_position", {3, solar_systems[i].sun_and_planets.pos_x.size()}, data);
        }
        {
            std::vector<double> data;
            for (double elem: solar_systems[i].asteroids.pos_x.vec()) {
                data.push_back(elem);
            }
            for (double elem: solar_systems[i].asteroids.pos_y.vec()) {
                data.push_back(elem);
            }
            for (double elem: solar_systems[i].asteroids.pos_z.vec()) {
                data.push_back(elem);
            }
            write_hdf5(group, "asteroids_position", {3, solar_systems[i].asteroids.pos_x.size()}, data);
        }
    }

}

/** N-body NICE simulation
 *
 * @param num_of_iterations  Number of iterations
 * @param num_of_planets     Number of planets
 * @param num_of_asteroids   Number of asteroids
 * @param seed               Random seed
 * @param filename           Filename to write each time step to. If empty, do data is written.
 */
void simulate(uint64_t num_of_iterations, uint64_t num_of_planets, uint64_t num_of_asteroids, uint64_t seed,
              const std::string &filename) {
    double dt = 1e12;
    double position_limit = 1e18;
    SolarSystem system = random_system(position_limit, num_of_planets, num_of_asteroids, seed);
    std::vector<SolarSystem> systems;
    systems.push_back(system);

    bhxx::flush();
    auto begin = std::chrono::steady_clock::now();
    for (uint64_t i = 0; i < num_of_iterations; ++i) {
        integrate(system, dt);
        if (!filename.empty()) {
            systems.push_back(system.copy());
        }
        bhxx::flush();
    }
    if (!filename.empty()) {
        write_hdf5(systems, filename);
    }
    auto end = std::chrono::steady_clock::now();
    std::cout << "elapsed time: " << (end - begin).count() / 1000000000.0 << " sec"<< std::endl;
}

/** Main function that parses the command line and start the simulation */
int main(int argc, char **argv) {
    util::ArgParser args(argc, argv);
    int64_t iterations, num_of_planets, num_of_asteroids, seed;
    if (args.cmdOptionExists("--iter")) {
        iterations = std::stoi(args.getCmdOption("--iter"));
        if (iterations < 0) {
            throw std::invalid_argument("iter most be a positive integer (e.g. --iter 100)");
        }
    } else {
        throw std::invalid_argument("You must specify the number of iterations (e.g. --iter 100)");
    }
    if (args.cmdOptionExists("--planets")) {
        num_of_planets = std::stoi(args.getCmdOption("--planets"));
        if (num_of_planets < 0) {
            throw std::invalid_argument("planets most be a positive integer (e.g. --planets 7)");
        }
    } else {
        throw std::invalid_argument("You must specify the number of planets (e.g. --planets 7)");
    }
    if (args.cmdOptionExists("--planets")) {
        num_of_planets = std::stoi(args.getCmdOption("--planets"));
        if (num_of_planets < 0) {
            throw std::invalid_argument("planets most be a positive integer (e.g. --planets 7)");
        }
    } else {
        throw std::invalid_argument("You must specify the number of planets (e.g. --planets 7)");
    }
    if (args.cmdOptionExists("--asteroids")) {
        num_of_asteroids = std::stoi(args.getCmdOption("--asteroids"));
        if (num_of_asteroids < 0) {
            throw std::invalid_argument("asteroids most be a positive integer (e.g. --asteroids 1000)");
        }
    } else {
        throw std::invalid_argument("You must specify the number of asteroids (e.g. --asteroids 1000)");
    }
    if (args.cmdOptionExists("--seed")) {
        seed = std::stoi(args.getCmdOption("--seed"));
        if (seed < 0) {
            throw std::invalid_argument("Seed most be a positive integer (e.g. --seed 42)");
        }
    } else {
        seed = std::random_device{}(); // Default seed is taken from hardware
    }
    const std::string &filename = args.getCmdOption("--out");

    simulate(static_cast<uint64_t>(iterations), static_cast<uint64_t>(num_of_planets),
             static_cast<uint64_t>(num_of_asteroids), static_cast<uint64_t>(seed), filename);
    return 0;
}



input_line_7:7:10: fatal error: 'H5Cpp.h' file not found
#include <H5Cpp.h>
         ^~~~~~~~~


Interpreter Error: 